In [2]:
!pip install qiskit qiskit-aer qiskit-ibm-runtime --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 95.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 145.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 88.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.4/377.4 kB 35.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 105.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.8/75.8 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 13.5 MB/s eta 0:00:00


In [4]:
# dj_tasks.py
# Qiskit Deutsch–Jozsa: Tasks 1–5

from __future__ import annotations
import os
from typing import Tuple, Dict
from dataclasses import dataclass

from qiskit import QuantumCircuit, transpile
from qiskit.quantum_info import Operator
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel, depolarizing_error, ReadoutError

# Optional (Task 4)
try:
    from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler
    IBM_RUNTIME_AVAILABLE = True
except Exception:
    IBM_RUNTIME_AVAILABLE = False


# ----------------------------
# Task 1 — Balanced Oracle (not just parity)
# f(x) = x0 XOR (x1 AND x2) XOR (x3 AND x4) XOR ...
# Implemented with only CX and CCX flipping the ancilla.
# ----------------------------
def balanced_oracle_xor_pairs(qc: QuantumCircuit, x_qubits, ancilla):
    """
    For n input qubits, flips ancilla according to:
        f(x) = x0 ^ (x1 & x2) ^ (x3 & x4) ^ ...
    This is balanced for any n>=1 over uniform inputs.
    Uses only CX (CNOT) and CCX (Toffoli).
    """
    n = len(x_qubits)
    if n == 0:
        return

    # Flip ancilla if x0 == 1
    qc.cx(x_qubits[0], ancilla)

    # For each pair (x_{2k+1}, x_{2k+2}), apply CCX onto ancilla to XOR in (x_i & x_{i+1})
    i = 1
    while i + 1 < n:
        qc.ccx(x_qubits[i], x_qubits[i+1], ancilla)
        i += 2

    # If we have a trailing single bit after pairing (when n is even, nothing to do; when odd, the trailing bit is already x0 covered)
    # We deliberately do NOT add the trailing single beyond x0, to keep the exact formula above.


# ----------------------------
# Deutsch–Jozsa circuit builder
# ----------------------------
@dataclass
class DJResult:
    n: int
    depth: int
    counts: Dict[str, int]


def build_dj_circuit(n: int, oracle_fn) -> QuantumCircuit:
    """
    Build a Deutsch–Jozsa circuit with n input (query) qubits and 1 ancilla.
    The oracle must flip the ancilla according to a Boolean function f(x).
    """
    qc = QuantumCircuit(n + 1, n)  # last qubit is ancilla, measure only input register
    x = list(range(n))
    anc = n

    # 1) Prepare |0...0>|1>
    qc.x(anc)

    # 2) Hadamards
    qc.h(x)
    qc.h(anc)

    # 3) Oracle U_f
    oracle_fn(qc, x, anc)

    # 4) Hadamards on input
    qc.h(x)

    # 5) Measure input only (ancilla not needed)
    qc.measure(x, list(range(n)))
    return qc


# ----------------------------
# Task 2 — Run for n = 2, 4, 5; report depth & outputs
# ----------------------------
def run_dj_simulator(n_values=(2, 4, 5), shots=2048) -> Dict[int, DJResult]:
    sim = AerSimulator()
    results = {}
    for n in n_values:
        qc = build_dj_circuit(n, balanced_oracle_xor_pairs)
        tqc = transpile(qc, sim)
        res = sim.run(tqc, shots=shots).result()
        counts = res.get_counts()
        results[n] = DJResult(n=n, depth=tqc.depth(), counts=counts)
    return results


# ----------------------------
# Task 3 — Noisy simulation
# ----------------------------
def make_simple_noise_model(p1=0.001, p2=0.01, ro_err=0.02) -> NoiseModel:
    """
    A small illustrative noise model:
      - Depolarizing error on 1- and 2-qubit gates
      - Symmetric 2-outcome readout error on each qubit
    """
    nm = NoiseModel()
    dep1 = depolarizing_error(p1, 1)
    dep2 = depolarizing_error(p2, 2)
    ro = ReadoutError([[1-ro_err, ro_err],
                       [ro_err, 1-ro_err]])

    # Apply to all basis gates (common: id, x, y, z, h, s, sdg, t, tdg, cx, ccx, etc.)
    # We at least cover single- and two-qubit gates typical in this circuit
    nm.add_all_qubit_quantum_error(dep1, ['x', 'h', 'z', 's', 'sdg', 't', 'tdg', 'rx', 'ry', 'rz', 'id'])
    nm.add_all_qubit_quantum_error(dep2, ['cx'])
    # (CCX will decompose to 1- and 2-qubit natives under transpile, so covered)
    nm.add_all_qubit_readout_error(ro)
    return nm


def run_dj_noisy(n_values=(2, 4, 5), shots=4096) -> Dict[int, DJResult]:
    nm = make_simple_noise_model()
    noisy_sim = AerSimulator(noise_model=nm)
    noisy_results = {}
    for n in n_values:
        qc = build_dj_circuit(n, balanced_oracle_xor_pairs)
        tqc = transpile(qc, noisy_sim, optimization_level=1)
        res = noisy_sim.run(tqc, shots=shots).result()
        counts = res.get_counts()
        noisy_results[n] = DJResult(n=n, depth=tqc.depth(), counts=counts)
    return noisy_results


# ----------------------------
# Task 4 — Run on IBM Quantum device (Runtime)
# ----------------------------
def run_on_ibm_quantum(n=4, backend_name: str | None = None, shots=4096) -> Dict[str, int]:
    """
    Requires:
      - pip install qiskit-ibm-runtime
      - An IBM Quantum account
      - Set env var QISKIT_IBM_TOKEN or call QiskitRuntimeService.save_account(...) once
    """
    if not IBM_RUNTIME_AVAILABLE:
        raise RuntimeError("qiskit_ibm_runtime not available. Install it to use Task 4.")

    # Initialize service: looks for env var or previously saved account
    token = os.environ.get("QISKIT_IBM_TOKEN")
    service = QiskitRuntimeService(
        channel="ibm_quantum",
        token=token if token else None
    )

    # Pick a backend
    if backend_name is None:
        # Choose a small, operational backend with enough qubits
        candidates = [
            b for b in service.backends()
            if b.configuration().n_qubits >= (n + 1) and b.status().operational
        ]
        if not candidates:
            raise RuntimeError("No suitable IBM Quantum backend found for the requested n.")
        # Naive least-busy heuristic
        backend = sorted(candidates, key=lambda b: b.status().pending_jobs)[0]
    else:
        backend = service.backend(backend_name)

    # Build and transpile
    qc = build_dj_circuit(n, balanced_oracle_xor_pairs)
    tqc = transpile(qc, backend, optimization_level=1)

    # Use Runtime Sampler for execution
    sampler = Sampler(backend=backend)
    job = sampler.run([tqc], shots=shots)
    result = job.result()
    # SamplerV2 returns quasi-dists; convert to counts-like dict (approximate by scaling)
    quasi = result[0].data.meas.get("quasi_dist", result[0].data.get("meas", {}))
    # Scale to "counts"
    counts = {k: int(round(v * shots)) for k, v in quasi.items()}
    return counts


# ----------------------------
# Task 5 — Circuit Analysis
# Print the oracle gate definition and (optionally) unitary for small n
# ----------------------------
def analyze_oracle(n=3, print_unitary=True):
    """
    Builds just the oracle on n inputs + ancilla and prints:
      - Gate definition of the oracle
      - (Optional) exact unitary (only feasible for small n)
    """
    qc = QuantumCircuit(n + 1)
    x = list(range(n))
    anc = n
    balanced_oracle_xor_pairs(qc, x, anc)

    # Print gate-level definition
    print("=== Oracle circuit (to_gate().definition) ===")
    print(qc.to_gate().definition)

    # Optionally show the full unitary (size 2^(n+1))
    if print_unitary and n <= 3:
        U = Operator(qc)
        print("\n=== Oracle unitary matrix ===")
        print(U.data)
    elif print_unitary:
        print("\n[Unitary omitted: size is 2^{} x 2^{} — too large to print here.]".format(n+1, n+1))


# ----------------------------
# Demo runner
# ----------------------------
def main():
    print("===== Task 2: Ideal simulator (n=2,4,5) =====")
    ideal = run_dj_simulator(n_values=(2, 4, 5), shots=2048)
    for n, res in ideal.items():
        print(f"\n[n={n}] depth={res.depth}")
        sorted_counts = sorted(res.counts.items(), key=lambda kv: kv[1], reverse=True)
        print("Counts:", dict(sorted_counts[:8]))

    print("\n===== Task 3: Noisy simulator (n=2,4,5) =====")
    noisy = run_dj_noisy(n_values=(2, 4, 5), shots=4096)
    for n, res in noisy.items():
        print(f"\n[n={n}] depth={res.depth} (noisy)")
        sorted_counts = sorted(res.counts.items(), key=lambda kv: kv[1], reverse=True)
        print("Counts:", dict(sorted_counts[:8]))

    print("\n===== Task 5: Oracle analysis (n=3) =====")
    analyze_oracle(n=3, print_unitary=True)



if __name__ == "__main__":
    main()


===== Task 2: Ideal simulator (n=2,4,5) =====

[n=2] depth=4
Counts: {'01': 2048}

[n=4] depth=5
Counts: {'0111': 524, '0011': 514, '0101': 510, '0001': 500}

[n=5] depth=6
Counts: {'10101': 146, '00011': 144, '11001': 142, '10111': 140, '10001': 135, '11011': 132, '00111': 130, '00001': 127}

===== Task 3: Noisy simulator (n=2,4,5) =====

[n=2] depth=4 (noisy)
Counts: {'01': 3906, '00': 114, '11': 74, '10': 2}

[n=4] depth=15 (noisy)
Counts: {'0001': 1018, '0101': 980, '0011': 967, '0111': 946, '0000': 39, '1011': 26, '0100': 25, '0010': 24}

[n=5] depth=23 (noisy)
Counts: {'00001': 298, '11101': 273, '00101': 268, '01101': 264, '00011': 259, '01111': 257, '10001': 253, '00111': 249}

===== Task 5: Oracle analysis (n=3) =====
=== Oracle circuit (to_gate().definition) ===
               
q_0: ──■───────
       │       
q_1: ──┼────■──
       │    │  
q_2: ──┼────■──
     ┌─┴─┐┌─┴─┐
q_3: ┤ X ├┤ X ├
     └───┘└───┘

=== Oracle unitary matrix ===
[[1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.